In [189]:
power_g = [15, 100, 32, 25, 70]
cost_g = [75, 15, 0, 42, 10]

power_d = [35, 23, 12, 38, 43, 16, 57]
cost_d = [65, 78, 10, 46, 63, 32, 50]

nd = len(power_d)
ng = len(power_g)
n = nd + ng

set_nd = range(1, nd+1)
set_ng = range(1, ng+1)

cg = {(i): cost_g[i-1] for i in set_ng}
cd = {(i): cost_d[i-1] for i in set_nd}

pg = {(i): power_g[i-1] for i in set_ng}
pd = {(i): power_d[i-1] for i in set_nd}

In [190]:
import pulp as plp

opt_model = plp.LpProblem(name="Model")

In [191]:
yg  = {(i): plp.LpVariable(cat=plp.LpContinuous, lowBound=0, upBound=pg[i], name="yg_{}".format(i)) for i in set_ng}
yd  = {(i): plp.LpVariable(cat=plp.LpContinuous, lowBound=0, upBound=pd[i], name="yd_{}".format(i)) for i in set_nd}

In [192]:
constraints = plp.LpConstraint(
                 e=plp.lpSum(yg[j] for j in set_ng) - plp.lpSum(yd[i] for i in set_nd),
                 sense=plp.LpConstraintEQ,
                 rhs=0,
                 name="constraint_eq")

In [193]:
opt_model += constraints, "balance between supply and demand"

In [194]:
objective = plp.lpSum(yg[i]*cg[i] for i in set_ng) - plp.lpSum(yd[j]*cd[j] for j in set_nd)

In [195]:
opt_model.sense = plp.LpMinimize

opt_model.setObjective(objective)

In [196]:
opt_model.solve()
#print(plp.LpStatus[opt_model.status])

1

In [197]:
import pandas as pd

opt_df = pd.DataFrame.from_dict(yd, orient="index", columns = ["variable_object"])

opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: item.varValue)
opt_df

,variable_object,solution_value
1,yd_1,35.0
2,yd_2,23.0
3,yd_3,0.0
4,yd_4,38.0
5,yd_5,43.0
6,yd_6,6.0
7,yd_7,57.0


In [198]:
import pandas as pd

opt_df = pd.DataFrame.from_dict(yg, orient="index", columns = ["variable_object"])

opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: item.varValue)
opt_df

,variable_object,solution_value
1,yg_1,0.0
2,yg_2,100.0
3,yg_3,32.0
4,yg_4,0.0
5,yg_5,70.0


In [199]:
for name, c in list(opt_model.constraints.items()):
    print(c.pi)

32.0
